data source: http://ai.stanford.edu/~amaas/data/sentiment/

In [1]:
import pyprind
import pandas as pd
import os

pbar = pyprind.ProgBar(50000)
labels = {'pos':1, 'neg':0}

df = pd.DataFrame()

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = 'aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r') as infile:
                text = infile.read()
                df=df.append([[text, labels[l]]], ignore_index=True)
                pbar.update()
                
df.columns = ['review', 'sentiment']
df.head(3)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:53


,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1


In [2]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))  # shuffle
df.to_csv('movie_data.csv', index=False)

In [3]:
df.sort_index(inplace=True)
df.head(3)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1


bag of words

1. a unique letter is a token, then vocabulary, like a meaningful word, consists of tokens.
2. create a vector including amount of words

In [4]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()  # count words frequency

docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'
])
bag = count.fit_transform(docs)

In [5]:
count.vocabulary_

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}

In [6]:
bag.toarray()

array([[0, 1, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1],
       [1, 2, 1, 1, 1, 2, 1]])

* CountVectorizer會計算單字出現在文件的次數
*  再透過 TfidfVectorizer轉換成TFIDF和IDF
* 也可以直接使用TfidfTransformer計算TFIDF

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer() # add weights, making some common words less important
np.set_printoptions(precision=2)  # l2 regulization
tfidf.fit_transform(bag).toarray()

array([[0.  , 0.43, 0.56, 0.56, 0.  , 0.43, 0.  ],
       [0.  , 0.43, 0.  , 0.  , 0.56, 0.43, 0.56],
       [0.4 , 0.48, 0.31, 0.31, 0.31, 0.48, 0.31]])

In [8]:
df.loc[1000, 'review']

'This is halfway to being a top movie. The opening section, which spoofs Hollywood "social message" films is absolutely brilliant. It is a riot from start to finish.<br /><br />The second section, which introduces us to the main characters of the story is really great too. We get a lot of great comic setups, top notch performances, and the dialog is really dynamic.<br /><br />(Spoiler warning!)<br /><br />The one think that really annoyed me about this film though is the ending, which I think contradicts everything that went before. My interpretation was that this film was taking the mickey out all the silly prejudices and innuendo of small town gossip and national tabloid sensationalism. I loved that the film was championing the cause that a person\'s sexuality is NOT determined by their hobbies, idiosyncrasies, fashion sense or whatever. And then the ending goes and re-enforces all the gossip and stereotypes that the movie successfully lampooned in the first place. It turns out every

In [9]:
import re

def preprocessor(text):
    
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [10]:
preprocessor(df.loc[1000, 'review'])

'this is halfway to being a top movie the opening section which spoofs hollywood social message films is absolutely brilliant it is a riot from start to finish the second section which introduces us to the main characters of the story is really great too we get a lot of great comic setups top notch performances and the dialog is really dynamic spoiler warning the one think that really annoyed me about this film though is the ending which i think contradicts everything that went before my interpretation was that this film was taking the mickey out all the silly prejudices and innuendo of small town gossip and national tabloid sensationalism i loved that the film was championing the cause that a person s sexuality is not determined by their hobbies idiosyncrasies fashion sense or whatever and then the ending goes and re enforces all the gossip and stereotypes that the movie successfully lampooned in the first place it turns out everyone was 100 right godamit this was very disappointing t

In [11]:
df['review'] = df['review'].apply(preprocessor) 

In [12]:
def tokenizer(text):
    return text.split()

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/burwei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.stem.porter import PorterStemmer  # translate words to stem, ex: running -> run
from nltk.corpus import stopwords

stop = stopwords.words('english')  # remove stopwords
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split() if word not in stop]

In [15]:
tokenizer_porter(df.loc[1000, 'review'])

['halfway',
 'top',
 'movi',
 'open',
 'section',
 'spoof',
 'hollywood',
 'social',
 'messag',
 'film',
 'absolut',
 'brilliant',
 'riot',
 'start',
 'finish',
 'second',
 'section',
 'introduc',
 'us',
 'main',
 'charact',
 'stori',
 'realli',
 'great',
 'get',
 'lot',
 'great',
 'comic',
 'setup',
 'top',
 'notch',
 'perform',
 'dialog',
 'realli',
 'dynam',
 'spoiler',
 'warn',
 'one',
 'think',
 'realli',
 'annoy',
 'film',
 'though',
 'end',
 'think',
 'contradict',
 'everyth',
 'went',
 'interpret',
 'film',
 'take',
 'mickey',
 'silli',
 'prejudic',
 'innuendo',
 'small',
 'town',
 'gossip',
 'nation',
 'tabloid',
 'sensation',
 'love',
 'film',
 'champion',
 'caus',
 'person',
 'sexual',
 'determin',
 'hobbi',
 'idiosyncrasi',
 'fashion',
 'sens',
 'whatev',
 'end',
 'goe',
 'enforc',
 'gossip',
 'stereotyp',
 'movi',
 'success',
 'lampoon',
 'first',
 'place',
 'turn',
 'everyon',
 '100',
 'right',
 'godamit',
 'disappoint',
 'actual',
 'great',
 'stori']

In [16]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
param = {
    'vect__tokenizer': [tokenizer, tokenizer_porter],
    'vect__stop_words': [stop, None],
    'vect__ngram_range': [(1, 1)],
    'clf__penalty': ['l1', 'l2'],
    'clf__C': [1.0, 10.0, 100.0]
   
}

In [18]:
lr_tfidf = Pipeline([
    ('vect', tfidf), ('clf', LogisticRegression(random_state=0))
])  # Tfidf Vectorizer first, then Logistic Regression

# gs_lr_tfidf = GridSearchCV(lr_tfidf, param, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
# gs_lr_tfidf.fit(X_train, y_train)

In [19]:
'Best parameter set: %s ' % gs_lr_tfidf.best_params_

NameError: name 'gs_lr_tfidf' is not defined

In [20]:
lr_tfidf = Pipeline([
    ('vect', TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, 
                             tokenizer=tokenizer, stop_words=None)), 
    ('clf', LogisticRegression(random_state=0, C=10.0, penalty='l2'))
])  

lr_tfidf.fit(X_train, y_train)
print('Test accuracy: %.3f' % lr_tfidf.score(X_train, y_train))

/Users/burwei/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test accuracy: 0.990


In [21]:
sklearn.metrics.accuracy_score(lr_tfidf.predict(X_test), y_test)

0.88268

In [22]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [23]:
next(stream_docs(path=r'movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [24]:
def get_minibath(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [25]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', n_features=21, preprocessor=None, 
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', n_iter_no_change=1)
doc_stream = stream_docs(path=r'movie_data.csv')

In [26]:
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])

for _ in range(45):
    X_train, y_train = get_minibath(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [27]:
X_test, y_test = get_minibath(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

Test accuracy: 0.594


In [29]:
import pickle

dest = os.path.join('movieclassifier', 'pkl_objects')

if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [35]:
cur_dir = os.getcwd()
stop = pickle.load(open(os.path.join(cur_dir,'movieclassifier', 'pkl_objects', 'stopwords.pkl'), 'rb'))

In [41]:
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, 
                         tokenizer=tokenizer)

In [42]:
example = ['I love this movie']
X = vect.transform(example)
clf.predict_proba(X)

ValueError: X has 2097152 features per sample; expecting 21